In [64]:
import sys,os
sys.path.append('../common')
from layers import *
from functions import softmax,cross_entropy_error  #softmax(活性化関数),エントロピー二乗誤差(誤差関数)
from gradient import numerical_gradient #numerical_gradient(パラメータの更新、勾配)import pprint
from collections import OrderedDict
import numpy as np

In [147]:
#重みWはガウス分布で初期化,バイアスは0で初期化
class Twolayer:
    
    def __init__(self,input_size,hidden_size,output_size,weight_init_std = 0.01):
        
        #重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size,hidden_size) #学習率(0.01)*ランダムに生成された行列(input_size×hidden_size)
        self.params['b1'] = np.zeros(hidden_size) #(1×hidden_size)、要素が0の1次元配列
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size,output_size) #行列の形が違う。
        self.params['b2'] = np.zeros(output_size) #行列の形が違う。
        
        #レイヤの生成
        self.layers = OrderedDict() #辞書作成
        self.layers['Affine1'] = Affine(self.params['W1'],self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'],self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
    
    def convert_t(self,t_train):
   
        t = np.zeros((t_train.shape[0],10))
        for i in range(t_train.shape[0]):
            label = t_train[i] #5
            t[i][label-1] = 1
        
        return t
        
    
    def predict(self,x): #layersを使うことでAffine,Relu,Affineと順序的に計算してくれる
        
        for layer in self.layers.values(): #layers.value()はlayersのリストの値を与えている。
            
            x = layer.forward(x) #Affine1,Relu,Affine2の順
            #print(x)
            
        return x
         
    
    def loss(self,x,t): #lossの算出 by cross_entropy_error
        y = self.predict(x)
        
        return self.lastLayer.forward(y,t) #Affine,Relu,Affineから得られた出力をsoftmax関数にかけて確率だしてそのエントロピー誤差を返す。
    
    def accuracy(self,x,t):
        
        y = self.predict(x)
        y = np.argmax(y,axis = 1)
        if t.ndim != 1 : t = np.argmax(t,axis = 1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self,x,t):
        loss_W = lambda W:self.loss(x,t) #入力と正解ラベルのlossを求める無名関数loss_Wの作成
        
        grads = {}
        
        grads['W1'] = numerical_gradient(loss_W,self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W,self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W,self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W,self.params['b2'])
        
        return grads
    
    def gradient(self,x,t): #入力、教師ラベル
        
        #forward:順伝搬では今まで通りlossを算出する。
        self.loss(x,t) #lossの中でpredictが呼ばれる。今、lossがわかっている。
        
        #backward:逆伝搬
        dout = 1
        dout  = self.lastLayer.backward(dout) #lastlayerはsmwlクラスのオブジェクト. smwlのbackwardの出力を返す。つまり、データ一個あたりの誤差。
        #print(dout)
        
        layers = list(self.layers.values()) #辞書layersの内容でリスト作成。[Affine1,Relu,Affine2]
        #print(layers)
        layers.reverse() #リストの要素を反転 [Affine2,Relu,Affine1]
        for layer in layers:
            dout = layer.backward(dout) #doutに各層(Affine2,Relu,Affine1)で微分した結果が入る。
            #print(dout)
        
        #print(dout.shape)
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads
        
        
        
        

In [148]:
import sys,os
sys.path.append('../')
from load_mnist import load_mnist
import numpy as np

In [150]:
x_train,t_train = load_mnist('',kind = 'train')
x_test,t_test = load_mnist('',kind = 't10k')

network = Twolayer(input_size=784,hidden_size=50,output_size=10)

t_train = network.convert_t(t_train)

x_batch = x_train[:3]
t_batch = t_train[:3]

#print(t_batch)

grad_backprop = network.gradient(x_batch,t_batch)

print(grad_backprop)







{'W1': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), 'b1': array([-0.00303995,  0.00202083,  0.        , -0.00619722, -0.00391892,
        0.        , -0.00198176,  0.00534743, -0.00474598,  0.0005082 ,
        0.00137362, -0.00816887,  0.00623957,  0.0082588 ,  0.        ,
       -0.01254076, -0.00207714,  0.00368835, -0.00512307,  0.        ,
       -0.0019353 ,  0.        ,  0.        ,  0.0009106 ,  0.        ,
       -0.00020548,  0.00181252, -0.00311134,  0.00203508, -0.00488136,
        0.00537629, -0.00024155,  0.00319916, -0.00039952,  0.00292047,
       -0.00245369, -0.00522642, -0.00233046,  0.00978083,  0.0005668 ,
        0.00629224,  0.00521449, -0.00730259,  0.00203109, -0.00682563,
        0.00287712,  0.00710373, -0.00134849, -0.00896313, -0.00126858]), 'W2': array([[ 1.581176